In [1]:
!pip install pandas sklearn transformers numpy torch pymorphy2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import json
from sklearn import tree
import pickle
from languageprocessor import MorphologyProcessor, SemanticsProcessor
from IPython.display import display
import numpy as np

In [3]:
prods=pd.read_csv('СТЕ.csv')
conts=pd.read_csv('Контракты.csv')

In [4]:
prods

,ID СТЕ,Название СТЕ,Категория,Код КПГЗ,Характеристики
0,1153097,мяч футбольный MIKASA REGATEADOR5-G,Мячи футбольные,01.08.01.13.01,"[{""Name"":""Модель"",""Id"":283795036,""Value"":""REGA..."
1,1153130,мяч волейбольный Gala Pro-Line 10 FIVB,Мячи волейбольные,01.08.01.14.03.01,"[{""Name"":""Марка"",""Id"":284249992,""Value"":""Gala""..."
2,1153163,мяч волейбольный Mikasa MVA380K-OBL,Мячи волейбольные,01.08.01.14.03.01,"[{""Name"":""Марка"",""Id"":284249802,""Value"":""MIKAS..."
3,1153174,мяч волейбольный Wilson Super Soft Play,Мячи волейбольные,01.08.01.14.03.01,"[{""Name"":""Марка"",""Id"":284246959,""Value"":""WILSO..."
4,1159300,Gutrend комплект расходных материалов для FUN ...,"Расходные материалы, комплектующие для прочего...",01.20.10.99,"[{""Name"":""Тип"",""Id"":284280400,""Value"":""Расходн..."
...,...,...,...,...,...
356570,35228990,Шина 23x10.50-12 107A8 Starco AS LOADER TL,"Шины для грузовых автомобилей и спецтехники, п...",01.09.08.10.05.02,"[{""Name"":""Бескамерные"",""Id"":369460372,""Value"":..."
356571,35229086,Кнопка включения рабочего освещения для минипо...,Запчасти к погрузчикам,01.09.08.03,"[{""Name"":""Артикул запчасти"",""Id"":369463362,""Va..."
356572,35231655,"Обои флизелиновые под покраску NC Antivandal, ...",Обои,01.11.03.11.08,"[{""Name"":""Описание"",""Id"":369399594,""Value"":""со..."
356573,35231695,Мусорное ведро Kimberly-Clark Aquarius белое п...,Контейнеры и другие емкости для мусора пластма...,01.20.03.03.06,"[{""Name"":""Вид емкости"",""Id"":369380812,""Value"":..."


In [5]:
conts

,Номер контракта,Дата публикации КС на ПП,Дата заключения контракта,Цена контракта,ИНН заказчика,КПП заказчика,Наименование заказчика,ИНН поставщика,КПП поставщика,Наименование поставщика,СТЕ
0,10/2019-СГЗ,13.06.2019 17:07,18.06.2019 0:00,21970,7709043455,770901001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕ...,5009062762,500901001.0,"ООО ""ОФИСМАГ""","[{""Id"":1226040,""Quantity"":100.00000,""Amount"":2..."
1,642-МЗ-44/19,13.06.2019 10:46,18.06.2019 0:00,85312,7719047184,771901001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,7725696613,772501001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МОСНАБ""","[{""Id"":null,""Quantity"":4.00000,""Amount"":42656...."
2,74-19,13.06.2019 15:34,18.06.2019 0:00,"26638,34",7727597777,772701001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,7724456672,772401001.0,"ООО ""Жизнь офиса""","[{""Id"":null,""Quantity"":1.00000,""Amount"":26638...."
3,И-1200/19,11.06.2019 15:07,18.06.2019 0:00,82386,7714338609,771401001,ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОС...,503224250310,NaN,Индивидуальный предприниматель Кручинина Екате...,"[{""Id"":1405814,""Quantity"":120.00000,""Amount"":8..."
4,639-МЗ-44/19,13.06.2019 10:55,18.06.2019 0:00,28362,7719047184,771901001,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,7706771569,772701001.0,"ООО ""ПРИНСИПАЛ-БМ""","[{""Id"":null,""Quantity"":300.00000,""Amount"":2836..."
...,...,...,...,...,...,...,...,...,...,...,...
414988,1536-17/40-Р3А,06.07.2017 11:36,10.07.2017 0:00,367220,7727063221,773601001,Государственное бюджетное общеобразовательное ...,9729011053,772901001.0,ООО «Фьюче Строй»,"[{""Id"":null,""Quantity"":1.00000,""Amount"":367220..."
414989,145/17,06.07.2017 10:45,10.07.2017 0:00,87020,7723169690,772301001,Государственное бюджетное общеобразовательное ...,7722631833,772201001.0,"Общество с ограниченной ответственностью ""Монт...","[{""Id"":null,""Quantity"":2.00000,""Amount"":87020...."
414990,1536-17/39-Р14А,05.07.2017 19:44,10.07.2017 0:00,312210,7727063221,773601001,Государственное бюджетное общеобразовательное ...,502499264403,NaN,Индивидуальный предприниматель Младин Василий,"[{""Id"":null,""Quantity"":1.00000,""Amount"":312210..."
414991,1557 АПС 0717,07.07.2017 15:40,10.07.2017 0:00,59740,7735093928,773501001,Государственное бюджетное общеобразовательное ...,7735534442,773501001.0,"Общество с ограниченной ответственностью ""Техн...","[{""Id"":null,""Quantity"":1.00000,""Amount"":59740...."


In [6]:
companies=conts["Наименование заказчика"].unique()
display(companies)

array(['ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ДОПОЛНИТЕЛЬНОГО ОБРАЗОВАНИЯ ГОРОДА МОСКВЫ «ДЕТСКАЯ МУЗЫКАЛЬНАЯ ШКОЛА ИМЕНИ Н.А.АЛЕКСЕЕВА»',
       'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХРАНЕНИЯ ГОРОДА МОСКВЫ «ГОРОДСКАЯ КЛИНИЧЕСКАЯ БОЛЬНИЦА ИМЕНИ Д.Д. ПЛЕТНЁВА ДЕПАРТАМЕНТА ЗДРАВООХРАНЕНИЯ ГОРОДА МОСКВЫ»',
       'ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ СПОРТИВНО-ДОСУГОВЫЙ ЦЕНТР «РАТМИР»',
       ...,
       'ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ГОРОДА МОСКВЫ «ПОКЛОННАЯ ГОРА»',
       'Государственное казенное учреждение города Москвы «Инженерная служба Тимирязевского района»',
       'ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ИНЖЕНЕРНАЯ СЛУЖБА ОБРУЧЕВСКОГО РАЙОНА»'],
      dtype=object)

In [7]:
products=prods["Категория"].unique()
size_prod=len(products)

In [8]:
sp=SemanticsProcessor()

In [9]:
recompute=False

In [10]:
if recompute:
    sp.load_pickle()
    company_vectors=dict()
    company_vectors_size=dict()
    compn=0
    for comp in companies:
        print(compn/len(companies))
        compn+=1
        vect=[]
        comp_conts=conts.loc[conts['Наименование заказчика'] == comp]
        #display(comp_conts)
        #display(comp_conts["СТЕ"])
        allcompvect=[]
        for cont in comp_conts["СТЕ"]:
            prod_info=json.loads(cont)[0]
            cte=(prods.loc[prods['ID СТЕ'] == prod_info["Id"]])
            prod_type=cte["Категория"].tolist()
            if not len(prod_type)==0:
                prod_type=prod_type[0]
                if comp not in company_vectors_size:
                    company_vectors_size[comp]=0
                company_vectors_size[comp]=company_vectors_size[comp]+1
                allcompvect.append(sp.get_embedding_precalc(prod_type).numpy())
        company_vectors[comp]=np.mean(np.array(allcompvect),axis=0)


In [11]:
import pickle as pkl

In [12]:
if recompute:
    saveFile = open("company_vectors.pickle", 'wb')
    pkl.dump((company_vectors,company_vectors_size), saveFile)

In [13]:
saveFile = open("company_vectors.pickle", 'rb')
(company_vectors,company_vectors_size) = pkl.load(saveFile)
print(len(company_vectors))
toRem=[]
for i in company_vectors:
    print(company_vectors[i])
    if np.isnan(np.max(company_vectors[i])):
        toRem.append(i)
for i in toRem:
    del company_vectors[i]

2457
[ 0.12152116 -0.21124959 -0.3231826  ...  0.12188186  0.48243845
  0.37746122]
[ 0.10637802 -0.24103145 -0.24308878 ...  0.09350092  0.46199113
  0.23693384]
[ 0.13117792 -0.25828516 -0.44067878 ... -0.09323788  0.24890514
  0.22561696]
[ 0.00990542 -0.28876483 -0.2594106  ...  0.06814366  0.4993135
  0.16635443]
[ 0.18684104 -0.19444144 -0.31749913 ...  0.07056814  0.30262506
  0.23431988]
[ 0.2203318  -0.22069743 -0.34445417 ...  0.1690669   0.28166747
  0.23002274]
[ 0.2005054  -0.19477135 -0.32735953 ...  0.14366172  0.30527112
  0.21618228]
[ 0.28531188 -0.19896509 -0.3152489  ...  0.12293077  0.335361
  0.2408826 ]
[ 0.23421152 -0.20915145 -0.3410956  ...  0.05065735  0.29976574
  0.24376905]
[ 0.19658583 -0.21621454 -0.2647426  ...  0.0835704   0.34286103
  0.22277339]
[ 0.25054055 -0.14303242 -0.38159013 ...  0.16923554  0.43396813
  0.22539799]
[ 0.15791203 -0.21643636 -0.314629   ...  0.10100025  0.31431076
  0.2530989 ]
[ 0.21627355 -0.22844248 -0.29583773 ...  0.055629

  0.25858662]
[ 0.16478798 -0.17499459 -0.28377733 ...  0.10474594  0.3192853
  0.24931437]
[ 0.27182984 -0.1869089  -0.3804642  ...  0.13024646  0.32568765
  0.22012712]
[ 0.20020409 -0.2175293  -0.36806127 ...  0.07397672  0.29317555
  0.2475922 ]
[ 0.25733337 -0.16780932 -0.27667028 ...  0.15160668  0.26287502
  0.24333124]
[ 0.19378681 -0.1866221  -0.36535293 ...  0.04603845  0.33610752
  0.2591838 ]
[ 0.22648115 -0.15759872 -0.46844056 ...  0.112621    0.33608437
  0.21569239]
[ 0.09876423 -0.22267018 -0.19594018 ...  0.08536789  0.21345568
  0.35607103]
[ 0.1904784  -0.15425602 -0.28734997 ...  0.1255266   0.2846135
  0.2433179 ]
[ 0.3568484  -0.22329813 -0.3337861  ...  0.17745872  0.3076047
  0.21501392]
[ 0.17258725 -0.22378823 -0.30245405 ...  0.0566869   0.28631747
  0.24895586]
[ 0.22429186 -0.1848932  -0.33173567 ...  0.09060812  0.32481414
  0.23477575]
[ 0.09190243 -0.22440006 -0.3425394  ...  0.09573455  0.45557508
  0.24451844]
[ 0.11375246 -0.26310474 -0.52158827 ... 

[ 0.17491774 -0.2117863  -0.29338744 ...  0.14352913  0.49633235
  0.12796408]
[ 0.20652424 -0.18204337 -0.32860893 ...  0.07265509  0.33457664
  0.2564077 ]
[ 0.19603159 -0.18408293 -0.3755082  ...  0.1322131   0.343269
  0.11913022]
[ 0.18173444 -0.2200014  -0.30778748 ...  0.04877646  0.2849452
  0.22671975]
[ 0.15023409 -0.2340721  -0.38856846 ...  0.07854056  0.25002837
  0.23527274]
[ 0.36007065 -0.16366751 -0.28928265 ...  0.22350207  0.35780218
  0.20289722]
[ 0.39144596 -0.10668533 -0.3288419  ...  0.2606567   0.48160604
  0.05586479]
[ 0.18506981 -0.20625871 -0.33263597 ...  0.09132674  0.50512165
  0.20099366]
[ 0.1253284  -0.22214824 -0.33323494 ...  0.07523014  0.34708288
  0.31995606]
[ 0.26898086 -0.19133651 -0.3430658  ...  0.08951011  0.31892282
  0.20186384]
[ 0.26725838 -0.18806334 -0.36300513 ...  0.10576437  0.33686084
  0.24022518]
[ 0.1526739  -0.24579668 -0.21255222 ...  0.12583677  0.41029444
  0.20015393]
[ 0.24637292 -0.20908298 -0.37850642 ...  0.15432379  0

[ 0.07232088 -0.14141339 -0.32852054 ...  0.11976893  0.08454162
  0.35944247]
[ 0.13879788 -0.11187106 -0.4449327  ...  0.15523314  0.5297081
  0.16620862]
[ 0.28484306 -0.1968673  -0.31711686 ...  0.09357016  0.24514417
  0.2020083 ]
[-0.01704085 -0.15643142 -0.43032777 ... -0.01638114  0.48528323
  0.4937742 ]
[ 0.2066249  -0.09342252 -0.18028143 ...  0.07324968 -0.01205914
  0.38747   ]
[ 0.4006135  -0.03106458 -0.51934266 ...  0.33033338  0.4347304
  0.24837944]
[ 0.3429887  -0.13645731 -0.2875558  ...  0.23262708  0.25891355
  0.20651875]
[ 0.03404524 -0.20694456 -0.3469524  ...  0.01137542  0.22631226
  0.32200456]
[ 0.0672963  -0.22129609 -0.45281312 ... -0.0027119   0.19126128
  0.26897666]
[ 0.3820438  -0.1435948  -0.29153746 ...  0.14067978  0.27960318
  0.23148979]
[ 0.12725215 -0.14898111 -0.44313458 ...  0.035843    0.31646913
  0.2432767 ]
[ 0.17967314 -0.20391053 -0.40436146 ...  0.0687107   0.37758768
  0.13967909]
[ 0.13458225 -0.1679078  -0.49365753 ...  0.14231023  

In [14]:
from scipy.spatial import distance
import datetime

In [44]:
compstr="ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ИНФОРМАЦИОННЫЙ ГОРОД»"
comp_conts=conts.loc[conts['Наименование заказчика'] == compstr]

cte_seq=[]
for cont in zip(comp_conts["СТЕ"],comp_conts["Дата публикации КС на ПП"]):
    prod_info=json.loads(cont[0])[0]
    cte=(prods.loc[prods['ID СТЕ'] == prod_info["Id"]])
    if len(cte["Категория"].tolist()):
        cte_seq.append([cte["Категория"].tolist(),datetime.datetime.strptime(cont[1], "%d.%m.%Y %H:%M")])
        
cte_seq.sort(key = lambda x: x[1])
#print(cte_seq[-1])
#cte_seq=cte_seq[0:len(cte_seq)-1]
display(cte_seq)

[[['Электрокофеварки бытовые'], datetime.datetime(2018, 11, 19, 16, 16)],
 [['Кресла офисные'], datetime.datetime(2018, 12, 3, 10, 17)],
 [['Папки пластиковые'], datetime.datetime(2018, 12, 3, 15, 8)],
 [['Холодильники бытовые'], datetime.datetime(2019, 2, 18, 12, 30)],
 [['Сейфы'], datetime.datetime(2019, 2, 19, 11, 8)],
 [['Пурифайер'], datetime.datetime(2019, 3, 11, 12, 12)],
 [['Шкафы хозяйственные металлические'],
  datetime.datetime(2019, 3, 25, 10, 15)],
 [['Средства чистящие для бытовой техники'],
  datetime.datetime(2019, 4, 8, 16, 53)],
 [['Файлы-вкладыши пластиковые'], datetime.datetime(2019, 4, 19, 11, 16)],
 [['Сейфы прочие'], datetime.datetime(2019, 6, 24, 10, 21)],
 [['Доски информационные'], datetime.datetime(2019, 6, 24, 14, 9)],
 [['Скамьи'], datetime.datetime(2019, 6, 25, 10, 25)],
 [['Термопоты бытовые'], datetime.datetime(2019, 7, 29, 12, 33)],
 [['Пурифайер'], datetime.datetime(2019, 7, 29, 14, 56)],
 [['Сейфы прочие'], datetime.datetime(2019, 8, 13, 10, 41)],
 [[

In [45]:
sp.load_pickle()
input_vals_text=[]
input_vals_vectors=[]
input_vals_data=[]

for i in cte_seq:
    input_vals_text.append(i[0][0])
    input_vals_data.append(i[1])

#display(input_vals_text)
for i in input_vals_text:
    print(i)
    input_vals_vectors.append(sp.get_embedding_precalc(i).numpy())

Электрокофеварки бытовые
Кресла офисные
Папки пластиковые
Холодильники бытовые
Сейфы
Пурифайер
Шкафы хозяйственные металлические
Средства чистящие для бытовой техники
Файлы-вкладыши пластиковые
Сейфы прочие
Доски информационные
Скамьи
Термопоты бытовые
Пурифайер
Сейфы прочие
Бланк бухгалтерский
Бумага для плоттеров и копировальных работ
Стеллажи складские металлические
Доски магнитно-маркерные
Элементы питания (батарейки)
Системы контроля доступа
Канаты, шнуры, тросы
Лента строительная адгезивная прочая
Оптические диски
Средства чистящие для бытовой техники
Фурнитура для инженерно-строительных товаров
Аптечки медицинские
Фурнитура для мебели металлическая
Фурнитура для мебели деревянная, пластиковая
Шуруповерты
Шкафы картотечные металлические
Инвентарь для санузлов пластмассовый
Оборудование для перевозки грузов
Энергосберегающие лампы
Вода питьевая
Инструмент мерительный электронный, оптический и электронно-оптический
Принадлежности гардеробные пластмассовые
Регистраторы подвесные
Нас

In [46]:
reference_company_vector=np.mean(input_vals_vectors, axis=0)

In [47]:
take_last=min(5,len(input_vals_vectors))
last_buys_vectors=input_vals_vectors[-take_last:]
reference_trajectory=[]
buy=last_buys_vectors[0]
for it in range(1,take_last):
    reference_trajectory.append((last_buys_vectors[it]-buy))
    buy=last_buys_vectors[it]

In [48]:
fit=dict()
for i in company_vectors:
    fit[i]=distance.euclidean(reference_company_vector, company_vectors[i])
fit={k: v for k, v in sorted(fit.items(), key=lambda item: item[1])}
max_fit=max(fit.values())
min_fit=min(fit.values())
print(max_fit,min_fit)
for i in fit:
    fit[i]=(1-((fit[i]-min_fit)/(max_fit-min_fit)))**20

13.969319343566895 2.828563992807176e-06


In [49]:
# OOOOOLLLLLLDDDDDDD
predict_vars_by_vect=dict()
normalize_vals=dict()
predict_vars=dict()

ind_num=0
for comp in fit:
    if ind_num>50 or fit[comp]<0.05:
        break
    ind_num+=1
    print(fit[comp],comp)
    comp_conts=conts.loc[conts['Наименование заказчика'] == comp]
    #display(comp_conts)
    #display(comp_conts["СТЕ"])
    allcompvect=[]
    allcomptypes=[]
    allcompdate_pre=(comp_conts["Дата публикации КС на ПП"].tolist())
    allcompdate=[]
    for dst in range(len(allcompdate_pre)):
        allcompdate_pre[dst]=datetime.datetime.strptime(allcompdate_pre[dst], "%d.%m.%Y %H:%M")
    didi_id=0
    for cont in comp_conts["СТЕ"]:
        prod_info=json.loads(cont)[0]
        cte=(prods.loc[prods['ID СТЕ'] == prod_info["Id"]])
        prod_type=cte["Категория"].tolist()
        if not len(prod_type)==0:
            #print(cont)
            prod_type=prod_type[0]
            allcompvect.append(sp.get_embedding_precalc(prod_type).numpy())
            allcomptypes.append(prod_type)
            allcompdate.append(allcompdate_pre[didi_id])
        didi_id+=1
    zip_data = list(zip(allcomptypes, allcompvect, allcompdate))
    zip_data = sorted(zip_data, key=lambda tup: tup[2])
    (allcomptypes,allcompvect,allcompdate)=zip(*zip_data)
    #print(allcompdate)
    sample_full_trajectory=[]
    sample_full_tr_date=[]
    for v_it in range(1,len(allcompvect)-1):
        sample_full_trajectory.append(allcompvect[v_it]-allcompvect[v_it-1])
        sample_full_tr_date.append(allcompdate[v_it]-allcompdate[v_it-1])
        
    
    for v_it in range(0,len(sample_full_trajectory)-take_last-1):
        sample_trajectory=sample_full_trajectory[v_it:v_it+take_last]
        pred_diff=sample_full_trajectory[v_it+take_last+1]
        pred_coord=input_vals_vectors[-1]+pred_diff
        pred_coord_ind=hash(str(sample_trajectory))
        #pred_cur=sp.choose_nearest_in_precalc_by_vector(pred_coord)
        #pred_cur=allcomptypes[v_it+take_last+1]
        if pred_coord_ind not in predict_vars_by_vect:
            predict_vars_by_vect[pred_coord_ind]=[[],0,0]
        if pred_coord_ind not in normalize_vals:
            normalize_vals[pred_coord_ind]=0
        normalize_vals[pred_coord_ind]+=1
        predict_vars_by_vect[pred_coord_ind][2]=sample_full_tr_date[v_it+take_last+1]
        predict_vars_by_vect[pred_coord_ind][0]=pred_coord
        for tr in range(len(reference_trajectory)):
            #print(distance.euclidean(reference_trajectory[tr],sample_trajectory[tr]))
            predict_vars_by_vect[pred_coord_ind][1]+=fit[comp]*distance.euclidean(reference_trajectory[tr],sample_trajectory[tr])
            #predict_vars[pred_cur]+=1*(1-fit[comp])



for i in predict_vars_by_vect:
    predict_vars_by_vect[i][1]/=normalize_vals[i]
    
predict_vars_by_vect={k: v for k, v in sorted(predict_vars_by_vect.items(), key=lambda item: item[1][1])}

#for i in predict_vars_by_vect:
#    print(predict_vars_by_vect[i])

ind_first=0
for vect in predict_vars_by_vect:
    if ind_first>50:
        break
    ind_first+=1
    predict_vars[sp.choose_nearest_in_precalc_by_vector(predict_vars_by_vect[vect][0])]=predict_vars_by_vect[vect][1:]
    
predict_vars={k: v for k, v in sorted(predict_vars.items(), key=lambda item: item[1])}
print("OK!")

1.0 ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ИНФОРМАЦИОННЫЙ ГОРОД»
0.13334617474546154 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ГОРОДА МОСКВЫ «ДОМ А.Ф. ЛОСЕВА - НАУЧНАЯ БИБЛИОТЕКА И МЕМОРИАЛЬНЫЙ МУЗЕЙ»
0.12584933912294868 Государственная инспекция по контролю за использованием объектов недвижимости города Москвы
0.12539766034898048 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ПЯТЬДЕСЯТ СЕДЬМАЯ ШКОЛА»
0.12216563599802677 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «МНОГОФУНКЦИОНАЛЬНЫЙ МИГРАЦИОННЫЙ ЦЕНТР»
0.11877332119807024 Государственное бюджетное учреждение культуры города Москвы «Централизованная библиотечная система Северо-Восточного административного округа»
0.11773738026241574 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ГОРОДА МОСКВЫ «МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ СИМФОНИЧЕСКИЙ ОРКЕСТР ДЛЯ ДЕТЕЙ И ЮНОШЕСТВА»
0.11471791947800324 префектура Северо-Восточного административного округа города Москвы
0.11315926795828153 Департамент торгов

In [50]:
display(predict_vars)

{'Домофоны': [3.5226143064129607, datetime.timedelta(days=2, seconds=85320)],
 'Прочие расходные материалы для культуры и искусства': [3.789851089876146,
  datetime.timedelta(seconds=1020)],
 'Унитазы': [3.8331942570809243, datetime.timedelta(days=2, seconds=77580)],
 'Дюбеля': [3.8476597274024833, datetime.timedelta(days=1, seconds=20400)],
 'Этикетки самоклеящиеся': [3.8476597274024833,
  datetime.timedelta(days=1, seconds=84120)],
 'Накопители твердотельные в формате флеш-карты': [3.8888251902176956,
  datetime.timedelta(seconds=1740)],
 'Инструкции по пожарной безопасности': [3.949871773894377,
  datetime.timedelta(seconds=4680)],
 'Разработка псд на монтаж оборудования': [3.963238204591203,
  datetime.timedelta(0)],
 'Переплетные машины': [3.9658832449000823, datetime.timedelta(seconds=56400)],
 'Плитки керамические': [4.033837266074623, datetime.timedelta(seconds=76980)],
 'Кровати детские': [4.06967129117023,
  datetime.timedelta(days=76, seconds=8460)],
 'Услуги дезинфекции': [

In [51]:
normalize_vals=dict()
predict_vars=dict()

ind_num=0
for comp in fit:
    if ind_num>10 or fit[comp]<0.05:
        break
    ind_num+=1
    print(fit[comp],comp)
    comp_conts=conts.loc[conts['Наименование заказчика'] == comp]
    #display(comp_conts)
    #display(comp_conts["СТЕ"])
    allcompvect=[]
    allcomptypes=[]
    allcompdate_pre=(comp_conts["Дата публикации КС на ПП"].tolist())
    allcompdate=[]
    for dst in range(len(allcompdate_pre)):
        allcompdate_pre[dst]=datetime.datetime.strptime(allcompdate_pre[dst], "%d.%m.%Y %H:%M")
    didi_id=0
    for cont in comp_conts["СТЕ"]:
        prod_info=json.loads(cont)[0]
        cte=(prods.loc[prods['ID СТЕ'] == prod_info["Id"]])
        prod_type=cte["Категория"].tolist()
        if not len(prod_type)==0:
            #print(cont)
            prod_type=prod_type[0]
            allcompvect.append(sp.get_embedding_precalc(prod_type).numpy())
            allcomptypes.append(prod_type)
            allcompdate.append(allcompdate_pre[didi_id])
        didi_id+=1
    zip_data = list(zip(allcomptypes, allcompvect, allcompdate))
    zip_data = sorted(zip_data, key=lambda tup: tup[2])
    (allcomptypes,allcompvect,allcompdate)=zip(*zip_data)
    #print(allcompdate)
    
    for v_it in range(0,len(allcomptypes)-take_last-1):
        #pred_coord=allcomptypes[v_it+take_last+1]
        #pred_coord_ind=hash(str(sample_trajectory))
        #pred_cur=sp.choose_nearest_in_precalc_by_vector(pred_coord)
        pred_text=allcomptypes[v_it+take_last]
        
        if pred_text not in predict_vars:
            predict_vars[pred_text]=[0,0]
        if pred_coord_ind not in normalize_vals:
            normalize_vals[pred_text]=0
        normalize_vals[pred_text]+=1
        
        predict_vars[pred_text][1]=allcompdate[v_it+take_last+1]-allcompdate[v_it+take_last]
        for tr in range(take_last):
            predict_vars[pred_text][0]+=distance.euclidean(allcompvect[tr+v_it],last_buys_vectors[tr])



for i in predict_vars:
    predict_vars[i][1]/=normalize_vals[i]
   

predict_vars={k: v for k, v in sorted(predict_vars.items(), key=lambda item: item[1])}
print("OK!")

1.0 ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ИНФОРМАЦИОННЫЙ ГОРОД»
0.13334617474546154 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ГОРОДА МОСКВЫ «ДОМ А.Ф. ЛОСЕВА - НАУЧНАЯ БИБЛИОТЕКА И МЕМОРИАЛЬНЫЙ МУЗЕЙ»
0.12584933912294868 Государственная инспекция по контролю за использованием объектов недвижимости города Москвы
0.12539766034898048 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «ПЯТЬДЕСЯТ СЕДЬМАЯ ШКОЛА»
0.12216563599802677 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ «МНОГОФУНКЦИОНАЛЬНЫЙ МИГРАЦИОННЫЙ ЦЕНТР»
0.11877332119807024 Государственное бюджетное учреждение культуры города Москвы «Централизованная библиотечная система Северо-Восточного административного округа»
0.11773738026241574 ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ КУЛЬТУРЫ ГОРОДА МОСКВЫ «МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ СИМФОНИЧЕСКИЙ ОРКЕСТР ДЛЯ ДЕТЕЙ И ЮНОШЕСТВА»
0.11471791947800324 префектура Северо-Восточного административного округа города Москвы
0.11315926795828153 Департамент торгов

In [52]:
datetime.datetime.strptime("14.09.2021 18:43", "%d.%m.%Y %H:%M")
display(predict_vars)

{'Пакеты полимерные': [57.43152141571045,
  datetime.timedelta(days=1, seconds=84720)],
 'Техническое обслуживание и текущий ремонт охранно-пожарных систем': [60.889835357666016,
  datetime.timedelta(days=1, seconds=78240)],
 'Набор карандашей цветных восковых (мелки)': [61.27827739715576,
  datetime.timedelta(seconds=1140)],
 'Расходные материалы и комплектующие для освещения': [65.90792751312256,
  datetime.timedelta(seconds=81180)],
 'Системное обслуживание (администрирование) информационных систем и ресурсов': [67.50727367401123,
  datetime.timedelta(days=14, seconds=2640)],
 'Оценка специальная условий труда': [67.65487098693848,
  datetime.timedelta(days=6, seconds=82740)],
 'Халат процедурный, одноразового использования': [69.28375911712646,
  datetime.timedelta(days=1, seconds=18660)],
 'Запасные части для системы видеонаблюдения': [69.82893180847168,
  datetime.timedelta(seconds=1680)],
 'Принадлежности и запасные части для клавишных инструментов': [69.84557819366455,
  dateti